In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.opts.defaults(hv.opts.Curve(width=500))

In [ ]:
import numpy as np
import scipy.signal
import scipy.fft

# Fuga espectral y técnica de enventanado

## Espectro de una señal limitada en el tiempo

Diremos que una señal es limitada en el dominio del tiempo si

$$
s(t) = 0 \quad \forall |t| > T,
$$

para alguna constante $T$

Diremos que una señal es limitada en el dominio de la frecuencia o de ancho de banda limitado si

$$
S(\omega) = 0 \quad \forall |\omega| > \Omega,
$$

para alguna constante $\Omega$

### Señal limitada como resultado de enventanado

Como vimos en la lección anterior, el espectro de 

$$
s(t) = \cos(2\pi f_0 t)
$$

es

$$
S(f) = \frac{1}{2} \left(\delta(f-f_0) + \delta(f+f_0) \right)
$$

Pero esto asume que $s(t)$ existen para todo $t$. 

:::{warning}
    
En la práctica siempre trabajamos con señales de duración finita

:::

Una versión "finita" y de duración $T>0$ de la señal anterior es equivalente a 

$$
s_T(t) = \cos(2 \pi f_0 t) \cdot \text{rect}(t/T),
$$

donde 

$$
\text{rect}(x) = \begin{cases} 1 & |x| \leq 1 \\ 0 & |x| > 0 \end{cases}
$$

se llama **ventana rectangular** o pulso cuadrado

In [ ]:
T = 5 # Duración de la ventana
Fs = 20 # Frecuencia de muestre
f0 = 1.2345 # Frecuencia fundamental
t = np.arange(-6, 6, step=1/Fs)
s = np.cos(2*np.pi*f0*t)
w = np.zeros_like(t)
w[np.absolute(t) < T/2] = 1
sT = s*w

In [ ]:
p1 = hv.Curve((t, s), 'Tiempo [s]', 'Señal', label='Original').opts(line_width=4)
p2 = hv.Curve((t, sT), 'Tiempo [s]', 'Señal', label='Enventanada')
p3 = hv.Curve((t, w), 'Tiempo [s]', 'Señal', label='Ventana').opts(line_dash='dashed', color='k')
(p1 * p2 * p3)

### Transformada de Fourier de la ventana rectangular

En la lección 2 vimos que la transformada de Fourier del pulso cuadrado es

$$
S(f) = \frac{T}{\pi f T} \sin(\pi f T) = T \text{sinc}(f T)
$$

donde la función sinc se define como

$$
\text{sinc}(x) = \frac{1}{\pi x} \sin(\pi x) 
$$

Puedes revisar la demostración de esta transformada [aquí](http://www.thefouriertransform.com/pairs/box.php)


El siguiente gráfico muestra como cambia $S(f)$ para distintos valores de $T$

In [ ]:
f = np.arange(-2, 2, step=1e-2)
S = {}
Ts = [2, 5, 10]
for T in Ts:
    S[T] = T*np.sinc(f*T)

In [ ]:
p1 = hv.Curve((f, S[2]), 'Frecuencia [Hz]', 'Espectro', label='T=2')
p2 = hv.Curve((f, S[5]), label='T=5')
p3 = hv.Curve((f, S[10]), label='T=10')
(p1 * p2 * p3)

:::{note}
    
Mientras mayor sea la duración temporal ($T$) de la ventana, más angosto y concentrado será su espectro

:::

### Transformada de Fourier de la señal enventanada

Usando la propiedad de modulación de la transformada de Fourier tenemos que

$$
\begin{align}
S_T(f) &= \mathbb{FT}[s_T(t)] \nonumber \\
&= \mathbb{FT}[\text{rect}_T(t) \cdot s(t) ] \nonumber \\
&=  \mathbb{FT}[\text{rect}_T(t)] * \mathbb{FT}[s_T(t)]    \nonumber \\
&=  T \text{sinc}(f T) *  \frac{1}{2} \left(\delta(f-f_0) + \delta(f+f_0) \right) \nonumber 
\end{align}
$$

:::{note}
    
Multiplicar por una ventana rectangular en el tiempo es equivalente a convolucionar con una función $\text{sinc}$ en frecuencia

:::


En la lección anterior estudiamos lo que significa convolucionar con un impulso. En este caso el resultado de la convolución es

$$
S_T(f) = \frac{T}{2} \left [ \text{sinc}((f - f_0)T) +  \text{sinc}((f + f_0)T) \right]
$$

Es decir un $\text{sinc}$ ubicado en la frecuencia fundamental, como muestra el siguiente diagrama

<img src="../images/sinc-conv.png">

Como vimos anteriormente el espectro del $\text{sinc}$ cambia con $T$. En general mientras más pequeño sea $T$ 

- más "corta" será  la señál enventanada $s_T(t)$ 
- más anchos serán los "lóbulos" de $S_T(f)$ 

El siguiente esquema muestra los lóbulos que tipicamente se observan en el espectro

<img src="../images/mainsidelobe.png" width="300">

- El lóbulo principal (*mainlobe* en la figura) es el que está relacionado a información real de la señal
- Los lóbulos laterales (*sidelobe* en la figura) son un efecto de la convolución con el **sinc**

:::{note}

La proporción entre el tamaño del lóbulo principal y los lóbulos laterales aumenta con $T$

:::

La siguiente figura muestra la señal coseno truncada con una ventana rectangular (columna izquierda) y su correspondiente espectro de amplitud (columna derecha). Observe como, a medida que $T$ disminuye, el lóbulo principal pierde definición

In [ ]:
Fs = 20 # Frecuencia de muestreo
f0 = 1.2345 # Frecuencia fundamental
t = np.arange(-10, 10, step=1/Fs)
s = np.cos(2*np.pi*f0*t)
f = scipy.fft.rfftfreq(n=len(s), d=1/Fs)

def rectangulo(t, T):
    w = np.zeros_like(t)
    w[np.absolute(t) < T/2] = 1.
    return w

p = []
for T in [15, 10, 5, 2]: # Duración de la ventana
    sT = s*rectangulo(t, T)
    ST = np.absolute(scipy.fft.rfft(sT))
    p.append(hv.Curve((t, sT), 'Tiempo [s]', 'Señal'))
    p.append(hv.Curve((f, ST), 'Frecuencia [Hz]', 'Espectro'))

hv.Layout(p).cols(2).opts(hv.opts.Curve(width=300, height=200))

## Fuga espectral

La aparición de los lóbulos laterales por truncamiento de la señal es un efecto que se conoce como **fuga espectral** (en inglés *spectral leak*)

La fuga espectral es una redistribución de la energía de un cierto componente espectral hacia sus frecuencias vecinas debido a las **discontinuidades o quiebres** en la periodicidad de la señal 

Observe a continuación la diferencia entre el espectro de una señal donde su periodicidad calza perfecto con la ventana (fila superior) versus cuando esto no ocurre, que sería el caso más general (fila inferior)

In [ ]:
Fs, T = 20, 2
t = np.arange(-T, T, step=1/Fs)
f = scipy.fft.rfftfreq(n=len(t), d=1/Fs)
f_os = scipy.fft.rfftfreq(n=len(t)*10, d=1/Fs)

p = []
for f0 in [2.0, 2.15]: # Frecuencia fundamental    
    sT = np.cos(2*np.pi*f0*t)    
    ST = np.absolute(scipy.fft.rfft(sT))   
    ST_os = np.absolute(scipy.fft.rfft(sT, n=len(sT)*10)) 
    
    p.append(hv.Curve((t, sT), 'Tiempo [s]', 'Señal'))
    p.append(hv.Curve((f, ST), 'Frecuencia [Hz]', 'Espectro') * hv.Curve((f_os, ST_os)).opts(alpha=0.25, color='k'))

hv.Layout(p).cols(2).opts(hv.opts.Curve(width=300, height=200))

- El inicio y término de la primera señal están conectados y su espectro tiene un lóbulo principal delgado (bien definido)
- El inicio y término de la segunda señal no es conectan y su espectro tiene un lóbulo principal grueso debido a la fuga de energía en sus vecinos


La fuga espectral es indeseada ya que no aporta información real. Podemos disminuir este efecto si hacemos que los bordes de la señal "calcen" antes de calcular su espectro. Este proceso de suavizado de bordes se llama **enventanado**


## Enventanado 

Es el proceso de multiplicar la señal por una ventana para alterar las características de su espectro. Lamentablemente no es posible eliminar la fuga espectral completamente con enventanado pero si podemos modificar como se redistribuye la energía

Existen muchas ventanas y cada una representa un compromiso (*trade-off*) distinto entre la <font color="0000BB">concentración o ancho del lóbulo principal</font> y la <font color="BB0000">atenuación de los lóbulos laterales</font>

En general se tiene que

- Mientras más abrupto es el corte de la ventana, más concentrado será el lóbulo principal y más fuertes serán los lóbulos laterales
- Mientras más suave sea la ventana, menos concentrado será el lóbulo principal y más débiles serán los lóbulos laterales

Es decir que siempre estamos sacrificando resolución por limpieza

Revisemos a continuación algunas ventanas (columna de la izquierda), su efecto al multiplicar con la señal (columna central) y en el espectro resultante (columna derecha)


In [ ]:
from scipy.signal import cosine, blackman, tukey

f0, Fs, T = 2.15, 20, 2
t = np.arange(-T, T, step=1/Fs)
sT = np.cos(2*np.pi*f0*t)
f_os = scipy.fft.rfftfreq(n=len(t)*10, d=1/Fs)
rect = lambda N: np.ones(shape=(N,))

p = []   
for window_fn in [rect, tukey, cosine, blackman]:
    w = window_fn(len(sT))    
    ST_os = np.absolute(scipy.fft.rfft(sT*w, n=len(sT)*10))    
    
    p.append(hv.Curve((t, w), 'Tiempo [s]', 'Ventana'))
    p.append(hv.Curve((t, sT*w), 'Tiempo [s]', 'Señal enventanada'))
    p.append(hv.Curve((f_os, ST_os), 'Frecuencia [Hz]', 'Espectro'))
    
hv.Layout(p).cols(3).opts(hv.opts.Curve(width=280, height=200))

Las ventanas espectrales más utilizadas están implementadas en el módulo [`scipy.signal`](https://docs.scipy.org/doc/scipy/reference/signal.windows.html?highlight=window#module-scipy.signal.windows)

En el ejemplo anterior usamos la ventana 

- Rectangular: Borde con discontinuidad fuerte, es la que tiene más concentración de lóbulo principal pero también más lóbulos laterales
- [Tukey](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.tukey.html#scipy.signal.windows.tukey) con parámetro $\alpha=0.5$, es una transición entre la ventana rectangular y la ventana coseno
- [Cosine](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.cosine.html#scipy.signal.windows.cosine), como su nombre lo indica tiene forma de coseno en $[-\pi/2, \pi/2]$
- [Blackman](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.blackman.html#scipy.signal.windows.blackman): Borde muy suave, es la que tiene menos concentración de lóbulo principal pero también menos lóbulos laterales


Otra ventana interesante es la ["ventana de Kaiser"](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.kaiser.html#scipy.signal.windows.kaiser) cuyo parámetro $\beta$ le permite asemejarse a otros ventanas de forma similar a la ventana de Tukey pero más flexible

- $\beta=0$: ventana rectangular
- $\beta=5$: similar a ventana coseno
- $\beta=8.8$ similar a la ventana de Blackman

Ejemplo a continuación

In [ ]:
from scipy.signal import kaiser

f0, Fs, T = 2.15, 20, 2
t = np.arange(-T, T, step=1/Fs)
sT = np.cos(2*np.pi*f0*t)
f_os = scipy.fft.rfftfreq(n=len(t)*10, d=1/Fs)

p = []    
for beta in [0, 5, 8.8]:
    w = kaiser(len(sT), beta=beta)    
    ST = np.absolute(scipy.fft.rfft(sT*w, n=len(t)*10))
    
    p.append(hv.Curve((t, w), 'Tiempo [s]', 'Ventana'))
    p.append(hv.Curve((t, sT*w), 'Tiempo [s]', 'Señal enventanada'))
    p.append(hv.Curve((f_os, ST_os), 'Frecuencia [Hz]', 'Espectro'))
    
hv.Layout(p).cols(3).opts(hv.opts.Curve(width=280, height=200))

### ¿Cómo escoger que ventana usar?

Esta es una pregunta sumamente importante pero difícil de responder ya que depende de cada problema

A continuación algunas indicaciones

- Para propósito general o como primera aproximación usar una ventana que entregue un buen compromiso, por ejemplo Cosine, Hamming o Hanning
- Si se requiere separar/discriminar frecuencias muy cercanas conviene usar una ventana que resalte más el lóbulo principal, por ejemplo rectangular o Tukey
- Si las frecuencias de interés están muy separadas entonces conviene usar una ventana que suprima más fuertemente los lóbulos laterales como la de Blackman

